<a href="https://colab.research.google.com/github/scalabrinig/cdProjetoAplicadoIV/blob/master/projeto/cd_projeto_aplicado_IV_entrega_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/joaonetto/projetoAplicado-IV/refs/heads/main/Imagem/Mackenzie.png" width="25%" align="left"/>



# **PROJETO APLICADO IV - Ciência de Dados EaD - 2026/01**


# **Entrega 1**

---

# **Titulo do Projeto**:

### *ChronoSec*: Detecção Comportamental e Séries Temporais para Segurança de Login (UEBA) em Eventos Google
---

In [1]:
#@title **Identificação do Grupo e Opção do Projeto**

#@markdown Integrantes do Grupo, nome completo em ordem alfabética (*informe: \<nome\>, \<matrícula\>*)
Aluno1 = 'João Silveira Campos Netto, 10441670' #@param {type:"string"}
Aluno2 = 'Alex Luiz Rabelo, 10442968' #@param {type:"string"}
Aluno3 = 'Antônio Henrique Caldas Mello, 10442968' #@param {type:"string"}

# **Introdução**

Processos de autenticação geram sinais ricos para segurança: falhas repetidas, desafios de MFA, mudanças de localização e padrões de horário. Em cenários de credenciais comprometidas, ataques automatizados e abuso de sessão, esses sinais costumam aparecer como anomalias temporais e comportamentais, muitas vezes antes de um incidente maior.

Este projeto propõe um produto analítico, implementado em Jupyter Notebooks, para explorar dados de login (pseudonimizado) provenientes da plataforma Google e aplicar técnicas de Séries Temporais e UEBA (User and Entity Behavior Analytics) para identificar desvios relevantes, como “Impossible Travel”, picos de frequência, sequências incomuns de eventos e mudanças no padrão de horário. A proposta combina regras explicáveis e modelos de aprendizado de máquina, com foco em reprodutibilidade e comunicação clara para um cliente (SOC/IAM).

UEBA é uma abordagem consolidada para detectar comportamentos anormais de usuários e entidades usando análises comportamentais e ML.



# **Motivações e justificativa**

### Motivação
1.	**Identidade virou o novo perímetro**

Grande parte dos incidentes modernos começa por abuso de credenciais (phishing, reutilização de senhas, token/session hijacking). O “momento do login” é um dos pontos mais ricos para detectar sinais iniciais de comprometimento, antes do dano maior.

2. **Logs de autenticação são séries temporais naturalmente**

Eventos de login têm ordem, frequência, dependência temporal e contexto (MFA, falha/sucesso, desafio), o que torna o problema ideal para técnicas de séries temporais e modelagem sequencial. Em vez de olhar eventos isolados, o projeto olha trajetórias de comportamento.

3. **Necessidade de reduzir ruído e acelerar triagem no SOC**

Times de segurança recebem muitos alertas, mas poucos são realmente relevantes. Um produto analítico que gere score + explicação ajuda a priorizar casos e reduzir tempo de investigação (MTTA/MTTR), com impacto direto na operação.

4. **“Impossible Travel” e padrões de sequência são sinais de alto valor**

Mudanças geográficas incompatíveis com o tempo, bursts de falhas e cadeias raras de autenticação são padrões amplamente utilizados na indústria, por serem interpretáveis e úteis na prática (mesmo antes do deep learning).

### Justificativa
1.	**Justificativa técnica: melhor modelar comportamento do que eventos pontuais**

Ataques de conta comprometida muitas vezes não são detectados por uma única característica. Eles aparecem como mudança de padrão: novo país/cidade, horário incomum, sequência anômala (falha→desafio→sucesso), ou velocidade impossível.

2. **Séries temporais (janelas, entropia, transições Markov, LSTM) são adequadas porque capturam:**
  - dependência temporal (antes/depois),
  - regularidade vs desvio (baseline do usuário),
  - anomalias contextuais (o que é normal para um usuário pode ser estranho para outro).

3. **Justificativa de negócio: prevenção e redução de impacto**

Detectar rapidamente abuso de credenciais reduz:
- fraudes e acessos indevidos,
- indisponibilidade por resposta a incidentes,
- custos de investigação e retrabalho,
- exposição reputacional e risco regulatório.

Além disso, gera um ativo “produto”: um motor de risco reaproveitável em outras aplicações (VPN, IAM, SSO, apps internos).

4. **Justificativa metodológica: abordagem em camadas (explicável + ML + deep learning)**

O projeto é justificável academicamente e operacionalmente porque entrega valor incremental:
- Regras explicáveis (rápidas de implementar e fáceis de defender em auditoria),
- Modelos clássicos (melhor generalização e redução de falsos positivos),
- Modelos sequenciais (Markov/LSTM) (capturam comportamento mais sutil).

Isso cria um caminho claro de maturidade do produto.

4. **Justificativa de dados: dataset apropriado e enriquecido**

Os dados anonimizados + geolocalização permitem medir:
- deslocamento (lat/lon + tempo),
- incerteza (accuracy_radius_km),
- contexto de autenticação (MFA/desafios/falhas),
- sinal auxiliar (É suspeito) para validação.

- Ou seja: há base suficiente para construir features robustas e validar hipóteses.

5. **Justificativa extensionista (ODS 9 e 12)**
- ODS 9: fortalecer resiliência e segurança de infraestrutura digital baseada em identidade.
- ODS 12: reduzir desperdício operacional (menos alertas inúteis, menos horas gastas, menos custos de resposta).

A entrega pública (código + dataset pseudonimizado + documentação) amplia o benefício para a comunidade acadêmica e profissionais de segurança.

# **Objetivo**

Este trabalho tem como objetivo desenvolver um produto analítico baseado em séries temporais para detectar e explicar comportamentos anômalos em processos de login (eventos de autenticação) a partir de dados anonimizados da plataforma Google enriquecidos com geolocalização. A pretensão central é transformar registros brutos de autenticação em informação acionável para Cyber Security, permitindo identificar rapidamente padrões compatíveis com abuso de credenciais, automação maliciosa e tentativas de acesso indevido, com foco em priorização operacional e redução de ruído na investigação.

Como metas, o projeto busca:
1. modelar o comportamento típico de usuários e domínios ao longo do tempo (baseline) e medir desvios relevantes;
2. implementar mecanismos de detecção para cinco dimensões complementares do problema: frequência/velocidade de tentativas (Velocity Checks), viagens impossíveis (Impossible Travel), sequências incomuns de eventos de autenticação via cadeias de Markov, mudanças no padrão de horário por entropia temporal, e aprendizado de comportamento por modelos LSTM;
3. consolidar essas evidências em um score de risco por evento acompanhado de justificativas interpretáveis, para apoiar triagem e tomada de decisão;
4. entregar um experimento executável e reproduzível em Jupyter Notebooks, incluindo pipeline de preparação de dados, extração de features, treinamento/avaliação de modelos e geração de relatórios com estudos de caso.

Ao final, espera-se disponibilizar uma solução que demonstre, de forma mensurável, a capacidade de identificar anomalias comportamentais e explicar por que um evento foi considerado suspeito, além de apresentar uma proposta extensionista alinhada às ODS 9 e 12 por meio da publicação do método e de materiais reprodutíveis (incluindo um dataset compatível com o esquema de dados), contribuindo para a comunidade interessada em segurança de identidade e análise temporal.



# **Descrição da base de dados**

Os dados utilizados neste projeto são provenientes de **streaming**, ou seja, consistem em logs gerados por evento. Dessa forma, a cada nova ação relevante (por exemplo, uma tentativa de autenticação), um novo registro é criado e pode ser coletado para análise e tomada de decisão em tempo quase real.mada de decisão, abaixo segue seu formato e sua descrição:

```json
{
  "Data": "2024-12-31T21:10:26-0300",
  "Usuário": "USR701910@acme-d67a0e6c.org.br",
  "Evento": "login_success",
  "Descrição": "USR701910@acme-d67a0e6c.org.br logged in",
  "Tipo de login": "google_password",
  "Tipo de desafio": "password",
  "Tipo de falha de login": "Desconhecido",
  "É suspeito": false,
  "É o segundo fator": false,
  "Nome da ação confidencial": null,
  "Domínio": "acme-d67a0e6c.org.br",
  "cidade": "São Paulo",
  "estado": "São Paulo",
  "país": "Brazil",
  "país ISO": "BR",
  "accuracy_radius_km": 20,
  "latitude": -23.5475,
  "longitude": -46.6361,
  "__geo_source": "mmdb",
  "__geo_error": null
}
```

### *Descrição dos campos*:

1) *Data*
- **O que é**: timestamp do evento, em ISO 8601 com timezone (ex.: -0300).
- **Para que serve**: no projeto: é a coluna central para séries temporais: janelas (5min, 1h, 24h), ordenação de sequência, cálculo de delta_t entre eventos e agregações por hora/dia/semana.

2) *Usuário*
- **O que é**: identificador do usuário anonimizado.
- **Para que serve**: chave para construir o baseline comportamental e sequências por usuário. Permite comparar *“o normal”* vs *“o desvio”* do próprio usuário.

3) *Evento*
- **O que é**: tipo do evento (ex.: login_success, login_failure, login_challenge, login_verification, logout, etc.).
- **Para que serve**: base para:
	- contagem e frequência,
	- cadeias de Markov e sequências,
	- modelagem LSTM,
	- caracterização de sessões (ex.: login→logout).

4) *Descrição*
- **O que é**: texto descritivo do evento (geralmente legível/humano).
- **Para que serve**: suporte para interpretação e auditoria (explicabilidade). Pode ser usado para validações, amostragens e exemplos no relatório.

5) *Tipo de login*
- **O que é**: mecanismo/canal de autenticação (ex.: google_password, reauth, exchange, Desconhecido).
- **Para que serve**: feature de contexto para risco; certos tipos podem ter perfis diferentes de ameaça e normalidade. Auxilia alerta.

6) *Tipo de desafio*
- **O que é**: tipo de desafio apresentado no fluxo (ex.: password, device_prompt, google_authenticator, security_key, none, etc.).
- **Para que serve**: sinaliza a “trilha” do login e suporte à análise sequencial e aprendizado. Também ajuda a medir fricção/segurança (ex.: desafios fortes vs fracos).

7) *Tipo de falha de login*
- **O que é**: categoria/motivo de falha de login.
- **Para que serve**: quando preenchido, seria essencial para classificar ataques (senha incorreta, conta bloqueada, etc.). Nesta base, aparece como “Desconhecido”, então o uso tende a ser limitado (mas o campo deve permanecer no schema para futura coleta/análise).

8) *É suspeito*
- **O que é**: flag (true/false) indicando se o evento já foi marcado como suspeito por alguma origem/regra prévia.
- **Para que serve**: útil como proxy de rótulo para validação inicial (ex.: avaliar precision/recall) e para análises de casos. Também pode ser usado como feature (com cautela, para não “vazar” decisão).

9) *É o segundo fator*
- **O que é**: flag (true/false) indicando ocorrência/uso de segundo fator.
- **Para que serve**: indicador de MFA; essencial para explicar e priorizar risco (ex.: “viagem impossível sem segundo fator”).

10) *Nome da ação confidencial*
- **O que é**: nome de uma ação sensível (quando aplicável) relacionada a risco/segurança.
- **Para que serve**: correlacionar eventos de autenticação com ações sensíveis (ex.: bloqueios, alterações de recovery, etc.), enriquecendo o score e estudos de caso.

11) *Domínio*
- **O que é**: domínio/tenant associado ao usuário e ao evento (também pseudonimizado).
- **Para que serve**: agregações por organização, deteção de surtos por domínio, e perfis distintos (um domínio pode ter comportamento diferente de outro).

---

### *Campos de geolocalização (enriquecimento)*

12) *cidade*
- **O que é**: cidade inferida via geolocalização de IP.
- **Para que serve**: contexto para “Impossible Travel”, clusters por cidade, baseline de locais frequentes e investigações.

13) *estado*
- **O que é**: estado/região administrativa.
- **Para que serve**: agregação intermediária (útil quando cidade é instável) e análises por região.

14) *país*
- **O que é**: país inferido.
- **Para que serve**: forte sinal para “Impossible Travel” e anomalias por deslocamento internacional.

15) *país ISO*
- **O que é**: código ISO do país (ex.: BR, US).
- **Para que serve**: padronização para relatórios, mapas e integração com outros sistemas.

16) *accuracy_radius_km*
- **O que é**: raio estimado de acurácia (km) da geolocalização (incerteza).
- **Para que serve**: redução de falsos positivos em velocidade/viagem impossível: deslocamentos pequenos podem ser ruído quando a acurácia é baixa; deslocamentos devem ser “descontados” pelo erro.

17) *latitude*
- **O que é**: latitude estimada.
- **Para que serve**: cálculo de distância (Haversine) e métricas de deslocamento, além de features para LSTM.

18) *longitude*
- **O que é**: longitude estimada.
- **Para que serve**: junto com latitude, cálculo de distância e construção de trajetórias.

19) *__geo_source*
- **O que é**: fonte do enriquecimento geo (ex.: mmdb, ipgeolocation, cache).
- **Para que serve**: qualidade e rastreabilidade do enrichment; fontes diferentes podem ter precisão distinta. Útil para filtros e explicações (“geo veio de cache/mmdb”).

20) *__geo_error*
- **O que é**: erro ocorrido durante o enriquecimento geo (quando existir).
- **Para que serve**: flag de qualidade; eventos com erro devem ser tratados como “geo inválido” (não usar em impossible travel/velocidade).

# **Referências (bibliográficas e técnicas)**

A seleção abaixo sustenta os principais pilares (UEBA, Impossible Travel, sequências com Markov e anomalia com LSTM):
1. UEBA (conceito e uso em detecção comportamental)
- IBM — [visão geral e definição de UEBA](https://www.ibm.com/think/topics/ueba).
- Rapid7 — [fundamentos de UEBA e relação com credenciais comprometidas e comportamento anômalo](https://www.rapid7.com/fundamentals/user-behavior-analytics).
2. Impossible Travel (prática de mercado / detecção)
- vCalc - [Haversine - Distance](https://www.vcalc.com/wiki/vcalc/haversine-distance)
- Google - [Detecting Impossible Travel, with Google SecOps: Part 1](https://security.googlecloudcommunity.com/community-blog-42/detecting-impossible-travel-with-google-secops-part-1-3892)
- Microsoft Defender for Cloud Apps — [política de anomalia e “impossible travel” com mitigação de falsos positivos](https://learn.microsoft.com/en-us/defender-cloud-apps/anomaly-detection-policy#impossible-travel).
- Microsoft Tech Community — [explicação do racional do “Impossible Travel”](https://techcommunity.microsoft.com/blog/microsoftthreatprotectionblog/detecting-and-remediating-impossible-travel/3366017).
3.	Modelagem sequencial com Markov para anomalia
- Ren (2017) — [Anomaly detection based on a dynamic Markov model](https://www.sciencedirect.com/science/article/pii/S0020025517307302).
- Haque (ACM, 2017) — [Markov Chain Modeling for Anomaly Detection in High Performance Computing System Logs](https://dl.acm.org/doi/epdf/10.1145/3152493.3152559).
4. LSTM/Autoencoders para detecção de anomalia em séries temporais
- Maleki (2021) — [Unsupervised anomaly detection with LSTM autoencoders using statistical data-filtering](https://www.sciencedirect.com/science/article/abs/pii/S1568494621003665).


In [2]:
#@title **Avaliação**
Introducao = 10 #@param {type:"slider", min:0, max:10, step:1}

Fonte_dos_dados = 10 #@param {type:"slider", min:0, max:10, step:1}

Solucao_proposta = 10 #@param {type:"slider", min:0, max:10, step:1}

Referencias = 10 #@param {type:"slider", min:0, max:10, step:1}

In [5]:
#@title **Nota Final**
nota = 0.30*Introducao + 0.30*Fonte_dos_dados + 0.30*Solucao_proposta + 0.10*Referencias

print(f'Nota da entrega {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_nome = []

for i in range(1,3):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_nome.append(lista[0]);")

alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota da entrega 10.0



,nome,nota
0,João Silveira Campos Netto,10.0
1,Alex Luiz Rabelo,10.0
